# Class 2: Early Language Modelling & Vector Representations

**Topics**

* N-gram-models
* Neural architectures
* TF-IDF
* Embeddings
* Applications of embeddings

**Libraries**:
NLTK, Gensim

**Reading**

* [Jurafsky & Martin Chapter 3: Ngram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)
* [Jurafsky & Martin Chapter 5:  Embeddings](https://web.stanford.edu/~jurafsky/slp3/5.pdf)



## 1. N-gram language models

In the next few cells we'll illustrate the strengths and limitations of n-gram-based language models by learning a 1-5-gram language model over Jane Austen's Persuasion using probabilities first calculated via Maximum Likelihood Estimation and then converted to log probabilities

We'll find that the perplexity of this language model is quite high, largely due to data sparsity -- many n-grams in the test set were not present in the training data, resulting in low or zero probabilities for these sequences in a our basic MLE model.

To address this, we'll implement smoothing techniques to mitigate the sparsity problem by assigning non-zero probabilities to unseen n-grams. This will give us a significantly lowe perplexity value.

In [ ]:
%pip install nltk
import nltk
nltk.download('punkt')
nltk.download('gutenberg')

## 1.1 Download and tokenize the data

We'll be using the NLTK library for our language modeling task. First we'll get the raw text of *Persuasion* and perform very basic whitespace-separated tokenization.

In [ ]:
from nltk.corpus import gutenberg
import re

# Load Persuasion
corpus = gutenberg.raw('austen-persuasion.txt')

# Tokenize the corpus by whitespace and convert to lowercase
tokens = re.findall(r'\b\w+\b', corpus.lower())

# Display the first few tokens
print(tokens[:20])

### 1.2 Create ngrams

We'll next create ngrams up to order 5--i.e,, unigrams, bigrams, etc--out of these tokens


In [ ]:
from nltk.util import ngrams

ngram_dict = {}
for n in range(1, 6):
    ngram_dict[n] = list(ngrams(tokens, n))

# Print the first few n-grams for each length
for n, ngram_list in ngram_dict.items():
    print(f"{n}-grams:")
    print(ngram_list[:10])
    print("-" * 20)

### 1.4 Maximum likelihood estimation

(See Jurafsky & Martin sec. 3.1.2.)
We use MLE to calculate the probabilty of an N-gram occurring based on its frequency in the training data as follows. The MLE probability of an N-gram $P(w_n | w_{n-N+1}...w_{n-1})$ is calculated as the count of the N-gram divided by the count of its context (the preceding N-1 words).

For example, the probability of a bigram $P(w_i | w_{i-1})$ is calculated as:

$P(w_i | w_{i-1}) = \frac{Count(w_{i-1} w_i)}{Count(w_{i-1})}$

Similarly, for a trigram $P(w_i | w_{i-2} w_{i-1})$:

$P(w_i | w_{i-2} w_{i-1}) = \frac{Count(w_{i-2} w_{i-1} w_i)}{Count(w_{i-2} w_{i-1})}$

In general, for an N-gram $w_{n-N+1}...w_n$, the MLE probability of the last word $w_n$ given the preceding N-1 words is:

$P(w_n | w_{n-N+1}...w_{n-1}) = \frac{Count(w_{n-N+1}...w_n)}{Count(w_{n-N+1}...w_{n-1})}$

Where:
- $Count(w_{n-N+1}...w_n)$ is the number of times the N-gram sequence appears in the training data.
- $Count(w_{n-N+1}...w_{n-1})$ is the number of times the context sequence (the first N-1 words of the N-gram) appears in the training data.


In [ ]:
from collections import Counter

ngram_probabilities = {}
ngram_counts = {}
context_counts = {}

for n in range(1, n + 1):
    ngram_counts[n] = Counter(ngram_dict[n])
    if n > 1:
        context_counts[n] = Counter(ngram[:-1] for ngram in ngram_dict[n])

for n in range(1, n + 1):
    ngram_probabilities[n] = {}
    for ngram, count in ngram_counts[n].items():
        if n == 1:
            # Probability for unigrams is count / total number of tokens
            ngram_probabilities[n][ngram] = count / len(tokens)
        else:
            # Probability for n-grams (n > 1) is count of n-gram / count of context
            context = ngram[:-1]
            if context in context_counts[n]:
                ngram_probabilities[n][ngram] = count / context_counts[n][context]
            else:
                #
                ngram_probabilities[n][ngram] = 0
# Display some of the calculated probabilities for each n-gram length
for n, probabilities in ngram_probabilities.items():
    print(f"{n}-gram probabilities (first 10):")
    for ngram, prob in list(probabilities.items())[:10]:
        print(f"{ngram}: {prob:.6f}")
    print("-" * 20)

### 1.5 Log Probabilities

As described in Jurafsky & Martin 3.1.3, multiplying many small probability values can lead to numerical underflow.  To avoid this, it is common practice to work with **log probabilities** instead of the raw probabilities. The logarithm of a product is the sum of the logarithms:

$\log(P(w_1, w_2, ..., w_m)) = \log(\prod_{i=1}^{m} P(w_i | w_{i-N+1}...w_{i-1}))$

Using the property of logarithms, this becomes:

$\log(P(w_1, w_2, ..., w_m)) = \sum_{i=1}^{m} \log(P(w_i | w_{i-N+1}...w_{i-1}))$



In [ ]:
import math

log_ngram_probabilities = {}

for n, probabilities in ngram_probabilities.items():
    log_ngram_probabilities[n] = {}
    for ngram, prob in probabilities.items():
        if prob > 0:
            log_ngram_probabilities[n][ngram] = math.log(prob)
        else:
            # Assign a very small negative number for log(0) to avoid errors
            # A common practice is to use a value related to the smallest possible non-zero probability or -infinity
            # Here, we'll use a placeholder like -1e10 for demonstration purposes,
            # but in a real application, more sophisticated smoothing techniques would be used.
            log_ngram_probabilities[n][ngram] = -float('inf') # Represent log(0) as negative infinity


# Display some of the calculated log probabilities for each n-gram length
for n, log_probabilities in log_ngram_probabilities.items():
    print(f"{n}-gram log probabilities (first 10):")
    # Safely iterate and print to avoid errors if a dictionary is empty
    for ngram, log_prob in list(log_probabilities.items())[:10]:
        print(f"{ngram}: {log_prob:.6f}")
    print("-" * 20)

### 1.6 Evaluate our model using the perplexity metric.

Perplexity is a measure of surprise so lower is better-- the mode is less surprised by the text.

Formally, the perplexity of a language model on a test set of $m$ words $W = w_1 w_2 ... w_m$ is defined as the inverse probability of the test set, normalized by the number of words:

$PP(W) = P(w_1 w_2 ... w_m)^{-\frac{1}{m}}$

Using the chain rule of probability, this can be rewritten as:

$PP(W) = (\prod_{i=1}^{m} P(w_i | w_1 ... w_{i-1}))^{-\frac{1}{m}}$

For an N-gram language model, the probability of a word $w_i$ is conditioned on the preceding N-1 words:

$PP(W) = (\prod_{i=1}^{m} P(w_i | w_{i-N+1} ... w_{i-1}))^{-\frac{1}{m}}$

To avoid numerical underflow when multiplying many small probabilities, we work with log probabilities:

$\log PP(W) = -\frac{1}{m} \sum_{i=1}^{m} \log P(w_i | w_{i-N+1} ... w_{i-1})$

And the perplexity is then:

$PP(W) = e^{-\frac{1}{m} \sum_{i=1}^{m} \log P(w_i | w_{i-N+1} ... w_{i-1})}$



In [ ]:
import math

# 1. Define a test set of tokens
# We'll use a portion of the existing tokens list.
# In a real scenario, this should be a separate dataset.
test_set_size = int(len(tokens) * 0.1) # Use 10% of the data as test set
test_tokens = tokens[-test_set_size:]
train_tokens = tokens[:-test_set_size] # Ensure training data is distinct

# Rebuild counts and probabilities using only the training data
# This is important for a proper evaluation on unseen data
ngram_counts_train = {}
context_counts_train = {}
log_ngram_probabilities_train = {}

for n in range(1, 6):
    train_ngrams = list(ngrams(train_tokens, n))
    ngram_counts_train[n] = Counter(train_ngrams)
    if n > 1:
        context_counts_train[n] = Counter(ngram[:-1] for ngram in train_ngrams)

for n in range(1, 6):
    log_ngram_probabilities_train[n] = {}
    for ngram, count in ngram_counts_train[n].items():
        if n == 1:
            prob = count / len(train_tokens)
        else:
            context = ngram[:-1]
            if context in context_counts_train[n]:
                prob = count / context_counts_train[n][context]
            else:
                prob = 0

        if prob > 0:
            log_ngram_probabilities_train[n][ngram] = math.log(prob)
        else:
            log_ngram_probabilities_train[n][ngram] = -float('inf')


# 2. Choose an N-gram length (e.g., N=5)
N = 5

# Generate N-grams for the test set
test_ngrams = list(ngrams(test_tokens, N))

# 3. Initialize variables
total_log_prob = 0
N_test = len(test_ngrams)

# Handle case where test set is smaller than N
if N_test == 0:
    perplexity = float('inf') # Cannot calculate perplexity
else:
    # 4. Iterate through the test set N-grams
    for ngram in test_ngrams:
        # a. Get the N-gram and its context
        # N-gram is 'ngram', context is 'ngram[:-1]' (for backoff, not strictly needed here)

        # b. Look up the log probability
        log_prob = log_ngram_probabilities_train[N].get(ngram, -float('inf')) # Use -inf for unseen ngrams

        # c. Add the log probability to total
        # Handle -inf by treating it as a very small probability (e.g., log(1e-10)) for sum
        # This prevents total_log_prob from becoming -inf immediately
        if log_prob == -float('inf'):
             total_log_prob += math.log(1e-10) # Using a small value as a proxy for unseen log prob
        else:
            total_log_prob += log_prob

    # 5. Calculate the average log probability
    avg_log_prob = total_log_prob / N_test

    # 6. Calculate perplexity
    # Handle potential overflow/underflow for very small/large avg_log_prob
    try:
        perplexity = math.exp(-avg_log_prob)
    except OverflowError:
        perplexity = float('inf')
    except UnderflowError:
        perplexity = 0.0 # Or a very small positive number

# 7. Print the calculated perplexity value
print(f"Calculating perplexity for N={N} model on the test set.")
print(f"Number of test N-grams: {N_test}")
print(f"Total log probability: {total_log_prob:.6f}")
print(f"Average log probability: {avg_log_prob:.6f}")
print(f"Perplexity: {perplexity:.4f}")

In [ ]:
print(f"Calculated Perplexity for N={N} model: {perplexity:.4f}\n")


Our perplexity is quite high. Given the relatively small corpus used and  the lack of smoothing, it's not too surprising that we performed so poorly on the test data:  a significant number of N-grams in the test set  were likely not seen in the training set (sparsity), leading to low or zero probabilities for these sequences. Let's try to fix this using smoothing.


### 1.7 Fix with smoothing techniques: Add-one and Add-*k* smoothing

See Jurafsky & Martin 3.3.
Without smoothing, the probability of an unseen N-gram is 0. This is problematic because it means any sequence containing an unseen N-gram will have a total probability of 0, leading to infinite perplexity.

Add-k smoothing adds a small constant $k$ to the count of every N-gram and adds $k$ times the vocabulary size to the denominator. This ensures that even unseen N-grams have a small, non-zero probability.

The formula for Add-k smoothing is:

$P_{Add-k}(w_n | w_{n-N+1}...w_{n-1}) = \frac{Count(w_{n-N+1}...w_n) + k}{Count(w_{n-N+1}...w_{n-1}) + k \times |V|}$

Where:
- $Count(w_{n-N+1}...w_n)$ is the number of times the N-gram sequence appears in the training data.
- $Count(w_{n-N+1}...w_{n-1})$ is the number of times the context sequence (the first N-1 words of the N-gram) appears in the training data.
- $k$ is the smoothing parameter (typically a small positive value, often between 0 and 1).
- $|V|$ is the size of the vocabulary (the total number of unique words in the training data).

When $k=1$, Add-k smoothing becomes Add-one smoothing.

In [ ]:
from collections import Counter
import math


# Add-one Smoothing function
def add_one_smoothing(n, ngram_counts, context_counts, vocab_size):
    smoothed_probabilities = {}
    for ngram, count in ngram_counts[n].items():
        if n == 1:
            smoothed_probabilities[ngram] = (count + 1) / (len(train_tokens) + vocab_size)
        else:
            context = ngram[:-1]
            # Add 1 to the count and the context count
            smoothed_probabilities[ngram] = (count + 1) / (context_counts[n].get(context, 0) + vocab_size)
    return smoothed_probabilities

# Add-k Smoothing function
def add_k_smoothing(n, ngram_counts, context_counts, vocab_size, k=1):
    smoothed_probabilities = {}
    for ngram, count in ngram_counts[n].items():
        if n == 1:
            smoothed_probabilities[ngram] = (count + k) / (len(train_tokens) + k * vocab_size)
        else:
            context = ngram[:-1]
            # Add k to the count and k * vocab_size to the context count
            smoothed_probabilities[ngram] = (count + k) / (context_counts[n].get(context, 0) + k * vocab_size)
    return smoothed_probabilities


# Calculate vocabulary size from the training data
vocab = set(train_tokens)
vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")

# Apply Add-one smoothing for N=5
N_smooth = 5
add_one_smoothed_probs = add_one_smoothing(N_smooth, ngram_counts_train, context_counts_train, vocab_size)

# Apply Add-k smoothing with k=1 for N=5 (equivalent to Add-one)
add_k_smoothed_probs_1 = add_k_smoothing(N_smooth, ngram_counts_train, context_counts_train, vocab_size, k=1)

# Apply Add-k smoothing with k=0.5 for N=5
add_k_smoothed_probs_half = add_k_smoothing(N_smooth, ngram_counts_train, context_counts_train, vocab_size, k=0.5)


print(f"\nDemonstrating Smoothing for N={N_smooth}:")
print("-" * 40)

# Example unseen N-gram from the previous perplexity calculation
unseen_ngram = ('hall', 'in', 'its', 'national', 'importance') # Example from previous output

print(f"Example N-gram: {unseen_ngram}")

mle_prob_unseen = 0
context_unseen = unseen_ngram[:-1]
if context_unseen in context_counts_train[N_smooth]:
    ngram_count_unseen = ngram_counts_train[N_smooth].get(unseen_ngram, 0)
    context_count_unseen = context_counts_train[N_smooth][context_unseen]
    mle_prob_unseen = ngram_count_unseen / context_count_unseen
else:
    mle_prob_unseen = 0 # Context not seen

print(f" Original MLE Probability (no smoothing): {mle_prob_unseen:.10f}")

# Probability with Add-one smoothing
add_one_prob_unseen = add_one_smoothed_probs.get(unseen_ngram, (0 + 1) / (context_counts_train[N_smooth].get(context_unseen, 0) + vocab_size)) # Handle unseen ngram in smoothed dict
print(f"  Add-one Smoothed Probability: {add_one_prob_unseen:.10f}")

# Probability with Add-k smoothing (k=1)
add_k_prob_unseen_1 = add_k_smoothed_probs_1.get(unseen_ngram, (0 + 1) / (context_counts_train[N_smooth].get(context_unseen, 0) + vocab_size)) # Handle unseen ngram in smoothed dict
print(f"  Add-k Smoothed Probability (k=1): {add_k_prob_unseen_1:.10f}")

# Probability with Add-k smoothing (k=0.5)
add_k_prob_unseen_half = add_k_smoothed_probs_half.get(unseen_ngram, (0 + 0.5) / (context_counts_train[N_smooth].get(context_unseen, 0) + 0.5 * vocab_size)) # Handle unseen ngram in smoothed dict
print(f"  Add-k Smoothed Probability (k=0.5): {add_k_prob_unseen_half:.10f}")


In [ ]:
import math

# Function to calculate perplexity with smoothed probabilities
def calculate_smoothed_perplexity(test_ngrams, smoothed_probabilities, N, context_counts_train, vocab_size, alpha=1):
    total_log_prob = 0
    N_test = len(test_ngrams)

    if N_test == 0:
        return float('inf')

    for ngram in test_ngrams:
        log_prob = -float('inf') # Default for unseen in smoothed dict

        if ngram in smoothed_probabilities:
             log_prob = math.log(smoothed_probabilities[ngram])
        else:
             # Calculate probability for unseen ngram in test set using smoothing formula
             context = ngram[:-1]
             if N == 1:
                  prob = (0 + alpha) / (len(train_tokens) + alpha * vocab_size)
             else:
                  prob = (0 + alpha) / (context_counts_train[N].get(context, 0) + alpha * vocab_size)

             if prob > 0:
                 log_prob = math.log(prob)
             else:
                 # This case should ideally not happen with smoothing and alpha > 0
                 log_prob = math.log(1e-10) # Fallback

        total_log_prob += log_prob

    avg_log_prob = total_log_prob / N_test

    try:
        perplexity = math.exp(-avg_log_prob)
    except OverflowError:
        perplexity = float('inf')
    except UnderflowError:
        perplexity = 0.0

    return perplexity

print(f"Recalculating Perplexity for N={N_smooth} with Smoothing:")
print("-" * 60)

# Calculate perplexity with Add-one smoothing
perplexity_add_one = calculate_smoothed_perplexity(test_ngrams, add_one_smoothed_probs, N_smooth, context_counts_train, vocab_size, alpha=1)
print(f"Perplexity with Add-one Smoothing: {perplexity_add_one:.4f}")

perplexity_add_k_half = calculate_smoothed_perplexity(test_ngrams, add_k_smoothed_probs_half, N_smooth, context_counts_train, vocab_size, alpha=0.5)
print(f"Perplexity with Add-k Smoothing (k=0.5): {perplexity_add_k_half:.4f}")

# Recall perplexity without smoothing from previous output
perplexity_no_smoothing = perplexity
print(f"Perplexity without Smoothing (MLE): {perplexity_no_smoothing:.4f}")

## 2. Sparse-vector representations: TF-IDF and term-document matrices

See Jurafsky & Martin 6.5.


One of the most common techniques for creating sparse vector representations is **TF-IDF (Term Frequency-Inverse Document Frequency)**. TF-IDF is a numerical statistic that reflects how important a word is to a document in a collection or corpus. It considers both the frequency of a word within a specific document (Term Frequency) and the rarity of the word across the entire corpus (Inverse Document Frequency).

**Term Frequency (TF)**

The Term Frequency of a word $t$ in a document $d$ is a measure of how often the word appears in that document. A simple way to calculate TF is:

$TF(t, d) = \text{Count of word } t \text{ in document } d$

However, it's often more useful to normalize the term frequency to prevent bias towards longer documents. Common normalization methods include:

*   **Raw Count**: $TF(t, d) = \text{Count}(t, d)$
*   **Adjusted for Document Length**: $TF(t, d) = \frac{\text{Count}(t, d)}{\text{Total number of words in document } d}$
*   **Log Normalization**: $TF(t, d) = \log(1 + \text{Count}(t, d))$
*   **Double Normalization**: $TF(t, d) = 0.5 + 0.5 \times \frac{\text{Count}(t, d)}{\text{Max frequency of any word in document } d}$

**Inverse Document Frequency (IDF)**

The Inverse Document Frequency of a word $t$ is a measure of how much information the word provides, i.e., if it's common or rare across all documents in the corpus. It is calculated as the logarithm of the total number of documents divided by the number of documents containing the word:

$IDF(t, D) = \log(\frac{\text{Total number of documents } |D|}{\text{Number of documents containing word } t})$

where $|D|$ is the total number of documents in the corpus. Adding 1 to the denominator (known as "smooth IDF") is common to prevent division by zero for words that don't appear in any document:

$IDF(t, D) = \log(\frac{|D|}{\text{Number of documents containing word } t + 1}) + 1$

**TF-IDF Calculation**

The TF-IDF score is the product of the Term Frequency and the Inverse Document Frequency:

$TF-IDF(t, d, D) = TF(t, d) \times IDF(t, D)$

A high TF-IDF score for a word in a document indicates that the word is frequent in that document but relatively rare in the rest of the corpus, making it a potentially important term for that document.

**Term-Document Matrix**

When we calculate the TF-IDF score for every word in the vocabulary for every document in the corpus, we can organize these scores into a **Term-Document Matrix**. In this matrix:

*   Each row represents a **term** (a unique word from the vocabulary).
*   Each column represents a **document** from the corpus.
*   Each cell $(i, j)$ contains the **TF-IDF score** of term $i$ in document $j$.

Alternatively, the matrix can be transposed, with rows representing documents and columns representing terms. This is the representation commonly produced by libraries like scikit-learn's `TfidfVectorizer`, where each row vector represents a document in the TF-IDF feature space.


In the next few cells, we'll learn some very basic TF-IDF representations for words in a handful of open-domain books. Using these learned representations we'll encode a set of documents and calculate their cosine similarity.

In [ ]:
import nltk
from nltk.corpus import gutenberg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# Download Gutenberg corpus if not already downloaded
try:
    nltk.data.find('corpora/gutenberg')
except nltk.downloader.DownloadError:
    nltk.download('gutenberg')
except LookupError:
    nltk.download('gutenberg')


# Load a few texts from the Gutenberg corpus
# Using 'austen-persuasion.txt' and 'shakespeare-hamlet.txt' as examples
corpus_texts = [
    gutenberg.raw('austen-persuasion.txt'),
    gutenberg.raw('austen-emma.txt'),
    gutenberg.raw('austen-sense.txt'),
    gutenberg.raw('shakespeare-caesar.txt'),
    gutenberg.raw('shakespeare-macbeth.txt'),
    gutenberg.raw('shakespeare-hamlet.txt'),
    gutenberg.raw('bible-kjv.txt') # Adding another text for more diversity
]

# Preprocess the texts (tokenize, lowercase, remove punctuation and numbers)
processed_corpus = []
for text in corpus_texts:
    # Tokenize by word, convert to lowercase
    tokens = re.findall(r'\b\w+\b', text.lower())
    # Join tokens back into a string for TfidfVectorizer
    processed_corpus.append(" ".join(tokens))

print(f"Loaded and processed {len(corpus_texts)} documents.")
print("-" * 50)

# Train the TfidfVectorizer
# We will treat each text as a document to build the vocabulary and calculate TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_corpus)

# Get the vocabulary (feature names)
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"TF-IDF Vectorizer trained. Vocabulary size: {len(feature_names)}")
print("-" * 50)


In [ ]:
documents_to_compare = [
    "quick brown fox jumps dog",
    "quick brown fox jumps cat",
    "quick brown fox jumps dog quick brown fox jumps cat",
    "I hate yellow cake"
]

In [ ]:
processed_documents_to_compare = []
for doc in documents_to_compare:
    tokens = re.findall(r'\b\w+\b', doc.lower())
    processed_documents_to_compare.append(" ".join(tokens))

# Transform the processed documents into TF-IDF vectors
# This uses the vocabulary and IDF learned from the training corpus
tfidf_matrix_compare = tfidf_vectorizer.transform(processed_documents_to_compare)

print("TF-IDF Vector Representations (Sparse Matrix):")
print(tfidf_matrix_compare)
print("-" * 50)
print(tfidf_matrix_compare.toarray())
print("-" * 50)

print("The TF-IDF vector representations of the four documents.")
print("Each row represents a document, and the columns correspond to the terms in the vocabulary learned from the training corpus.")
print("The values are the TF-IDF scores, indicating the importance of each term in each document.")


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate the cosine similarity matrix
cosine_sim_matrix_compare = cosine_similarity(tfidf_matrix_compare)
print("Cosine Similarity Matrix:")
print(cosine_sim_matrix_compare)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cosine_sim_matrix_compare, annot=True, cmap='viridis', fmt=".2f",
                xticklabels=[f"Doc {i+1}" for i in range(len(documents_to_compare))],
                yticklabels=[f"Doc {i+1}" for i in range(len(documents_to_compare))])
plt.title("TF-IDF Cosine Similarity Heatmap between Documents")
plt.xlabel("Document")
plt.ylabel("Document")
plt.show()

## 3. Neural network language models and word2vec

**Word2Vec: Learning Dense Word Representations**

See Jurafsky & Martin 6.8. While TF-IDF provides sparse vector representations of words, Word2Vec is a popular algorithm for learning **dense** vector representations of words, also known as **word embeddings**. These embeddings capture semantic relationships between words -- words with similar meanings will lie close to each other in a  continuous vector space.

We will focus on the **Skip-Gram** model in this illustration.

**The Skip-Gram Procedure**

The Skip-Gram model works by taking pairs of words from the training text: a **target word** (the current word) and **context words** (words within a defined window around the target word). The goal is to train a model that is good at predicting context words given a target word.

For each target word in the corpus, the Skip-Gram model samples context words within a specified window size. For example, with a window size of 2, for the sentence "the quick brown fox jumps over the lazy dog", when "fox" is the target word, the context words would be "quick", "brown", "jumps", and "over".

The model uses a simple two-layer neural network (without a hidden layer in its original form) to learn the word embeddings.

*   **Input Layer**: Takes the target word as a one-hot encoded vector.
*   **Output Layer**: Outputs a probability distribution over the vocabulary, representing the likelihood of each word being a context word for the given target word.

The core idea is that if two words have similar contexts (i.e., they appear in similar surrounding words), their learned vector representations should be close in the vector space.

Word2Vec is considered a "self-training" or "self-supervised" algorithm because it learns word embeddings from the text data itself without requiring external labeled data. The training objective is derived directly from the structure of the input text.

The process involves:

1.  **Creating Word Pairs**: Generating (target word, context word) pairs from the training corpus based on the chosen window size.
2.  **Training the Model**: The model is trained to maximize the probability of predicting the actual context words for a given target word. This is typically done using a technique called **Negative Sampling**. Instead of calculating the probability of all words in the vocabulary, it samples a small number of "negative" words (words that are *not* in the context) and trains the model to distinguish the positive context words from the negative samples.
3.  **Learning Embeddings**: As the model trains to perform this prediction task, the weights learned in the "projection layer" (which effectively maps the one-hot input to a dense vector) become the word embeddings. These dense vectors capture the distributional information of words in the corpus.s.

In [ ]:
%pip install gensim

In [ ]:
import gensim.downloader as api
import numpy as np

# Load a pre-trained Word2Vec model
# This might take some time and download a large file (around 1.5 GB for word2vec-google-news-300)
print("Loading pre-trained Word2Vec model...")

model = api.load("word2vec-google-news-300")

In [ ]:
word_to_query = "computer"
similar_words = model.most_similar(word_to_query, topn=50)
# Print the similar words and their similarity scores
for word, similarity in similar_words:
  print(f"{word}: {similarity:.4f}")